In [25]:
!pip install duckdb

In [26]:
import os
import yaml
import time 
import duckdb
import openai
import pandas as pd

In [27]:
with open('cadentials.yaml') as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

os.environ['OPENAI_API_KEY'] = credentials['OPENAI_API_KEY']
os.environ['HUGGINGFACEHUB_API_TOKEN'] = credentials['HUGGINGFACEHUB_API_TOKEN']
os.environ['ENGINE'] = credentials['ENGINE']

openai.api_key = credentials['OPENAI_API_KEY']
openai.api_base = credentials['OPENAI_API_BASE']
openai.api_type = credentials['OPENAI_API_TYPE']
openai.api_version = credentials['OPENAI_API_VERSION']
openai.engine = credentials['ENGINE']

In [28]:
def load_data(data_path):
    df = pd.read_csv(
                    data_path,
                    sep=';'
                    )
    return df

In [29]:
df = load_data('datasets/Banking-Marketing Targets/train.csv')
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [58]:
def openai_completion(
                    query_prompt,
                    table,
                    col_names
                    ):
    response = openai.ChatCompletion.create(
                                            engine = os.environ["ENGINE"],
                                            model="gpt-3.5-turbo",
                                            messages=[
                                                    {
                                                    "role": "system", 
                                                    "content": """
                                                                    Given the following SQL table, your job is to write queries given a user's request. \n

                                                                    CREATE TABLE {} ({})
                                                               """.format(table, col_names)
                                                    },
                                                    {
                                                    "role": "user", 
                                                    "content": query_prompt
                                                    },
                                                ]
                                            )
    return response['choices'][0]['message']['content']

### try a sample

In [59]:
table = "banking_targets"
col_names = str(list(df.columns)).replace('[', '').replace(']', '')
query = "Average age of provided records ?"

In [60]:
sql_response = openai_completion(query, table, col_names)
sql_response

'SELECT AVG(age) FROM banking_targets'

In [2]:
duckdb.sql(sql_response).show()